In [102]:
import h5py
import pandas as pd
import SimpleITK as sitk
from skimage.segmentation import clear_border
from skimage import morphology
from skimage import measure
from skimage import filters
from scipy.ndimage.morphology import binary_fill_holes
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [111]:
def get_segmented_lungs(im, plot=False, THRESHOLD = -320):

    binary = im < THRESHOLD

    cleared = clear_border(binary)

    label_image = measure.label(cleared)

    areas = [r.area for r in measure.regionprops(label_image)]
    areas.sort()
    #print areas
    if len(areas) > 2:
        for region in measure.regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                    label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0

    selem = morphology.disk(2)
    binary = morphology.binary_erosion(binary, selem)
    
    selem = morphology.disk(15)
    binary = morphology.binary_closing(binary, selem)
    
    edges = filters.roberts(binary)
    binary = binary_fill_holes(edges)
   
    get_high_vals = binary == 0
    im[get_high_vals] = 0

    return im

In [120]:
cd /Share/home/chenxupeng/projects/pr/

/Share/home/chenxupeng/projects/pr


In [112]:
id = np.loadtxt('preprocess/patient_id',dtype = 'S')

In [108]:
id_ = {}
for i in range(30):
    id_[i] = id[20*i:20*(i+1)]

In [110]:
for i in range(30):
    np.savetxt('preprocess/patient_id_'+str(i+1),id_[i],fmt = '%s')

In [63]:
processed_array = {}
for j in range(2):
    with h5py.File('img_array','r') as f:
        test = f[id[j]][:,:,:]
    tensor_dim1 = test.shape[0] 
    processed_array[j] = {}
    for i in range(tensor_dim1):
        processed_array[j][i] = get_segmented_lungs(test[i], THRESHOLD = -320)

In [ ]:
600 * 300 * 矩阵

In [94]:
tt = {}
for i in range(2):
    tt[i] = np.array([val for (key,val) in processed_array[i].iteritems()])

In [100]:
with h5py.File('processed_array','w') as f:
    for i in range(2):
        f.create_dataset(id[i],data = tt[i])

In [127]:
k = 4
processed_array = {}
with h5py.File('preprocess/img_array_'+str(k),'r') as f:
    for j in range(20*(k-1),20*k):
        test = f[id[j]][:,:,:]
        tensor_dim1 = test.shape[0]
        processed_array[j] = {}
        for i in range(tensor_dim1):
            processed_array[j][i] = get_segmented_lungs(test[i], THRESHOLD = -320)
        print 'first step'
        
processed_tensor = {}
for i in range(20*(k-1),20*k):
    processed_tensor[i] = np.array([val for (key,val) in processed_array[i].iteritems()])

print 'second step'
with h5py.File('preprocess/processed_tensor_'+str(k),'w') as f:
    for i in range(20*(k-1),20*k):
        f.create_dataset('preprocess'+id[i],data = processed_tensor[i])
        
print 'third step'

KeyboardInterrupt: 

In [124]:
processed_array = {}
with h5py.File('preprocess/img_array','r') as f:
    for j in range(20,40):
        test = f[id[j]][:,:,:]
        tensor_dim1 = test.shape[0]
        processed_array[j] = {}
        for i in range(tensor_dim1):
            processed_array[j][i] = get_segmented_lungs(test[i], THRESHOLD = -320)
        print 'first step'
        
processed_tensor = {}
for i in range(20,40):
    processed_tensor[i] = np.array([val for (key,val) in processed_array[i].iteritems()])

print 'second step'
with h5py.File('preprocess/processed_tensor_2','w') as f:
    for i in range(20,40):
        f.create_dataset('preprocess'+id[i],data = processed_tensor[i])
        
print 'third step'

first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
first step
second step
third step


In [131]:
processed_array[60][0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [ ]:
processed_array = {}
with h5py.File('preprocess/img_array','r') as f:
    for j in range(2):
        test = f[id[j]][:,:,:]
        tensor_dim1 = test.shape[0]
        processed_array[j] = {}
        for i in range(tensor_dim1):
            processed_array[j][i] = get_segmented_lungs(test[i], THRESHOLD = -320)
print 'first step'
        
processed_tensor = {}
for i in range(2):
    processed_tensor[i] = np.array([val for (key,val) in processed_array[i].iteritems()])

print 'second step'
with h5py.File('preprocess/processed_tensor_test_1','w') as f:
    for i in range(2):
        f.create_dataset('preprocess'+id[i],data = processed_tensor[i])
        
print 'third step'

In [68]:
'''
with h5py.File('img_array','r') as f:
    test = f['LKDS-00001'][:,:,:]
tensor_dim1 = test.shape[0] 
processed_array = {}
for i in range(tensor_dim1):
    processed_array[i] = get_segmented_lungs(test[i], THRESHOLD = -320)
'''

"\nwith h5py.File('img_array','r') as f:\n    test = f['LKDS-00001'][:,:,:]\ntensor_dim1 = test.shape[0] \nprocessed_array = {}\nfor i in range(tensor_dim1):\n    processed_array[i] = get_segmented_lungs(test[i], THRESHOLD = -320)\n"

In [22]:
'''
path = np.loadtxt('patient_path_unique',dtype = 'S')
id_ = {}
for i in range(600):
    id_[i] = path[i].split('/')[2].split('.')[0]
id = np.array([val for (key,val) in id_.iteritems()])
np.savetxt('patient_id',id,fmt = '%s')
'''

"\npath = np.loadtxt('patient_path_unique',dtype = 'S')\nid_ = {}\nfor i in range(600):\n    id_[i] = path[i].split('/')[2].split('.')[0]\nid = np.array([val for (key,val) in id_.iteritems()])\nnp.savetxt('patient_id',id,fmt = '%s')\n"